In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
df_trec = pd.read_parquet("../../../data/metadata_TREC.parquet")

In [3]:
# Read iso file to resolve country name abbrevations

with open("../../../data/iso3166-1.json", 'r', encoding="utf-8") as file:
    Country_codes = json.load(file)


country_codes_new = {}
for i in Country_codes["3166-1"]:
    country_codes_new[i["alpha_2"]] = i["name"]

In [4]:
# Extract metadata informations from OpenAlex regarding author names, institution names and location of the institution

def calculate_most_active_countries_institutions(country_codes_dict,df,path):
    with open(path, 'r', encoding="utf-8") as file:
        OpenAlexJson = json.load(file)

    countries_list = []
    for i in OpenAlexJson:
        country_codes = []
        institutions = []
        for j in OpenAlexJson[i]["authorships"]:
            for k in j["institutions"]:
                if k["country_code"] is not None and k["display_name"] is not None:
                    if [country_codes_new[k["country_code"]], k["display_name"], j["author"]["display_name"]] not in country_codes:
                        country_codes.append([country_codes_new[k["country_code"]], k["display_name"], j["author"]["display_name"]])
        countries_list.append([i, country_codes])

    df_countries = pd.DataFrame(countries_list, columns=["ID", "Countries"])
    df = pd.merge(df, df_countries, how ="inner", right_on="ID",left_on="ID")
    return df


In [6]:
df_trec = calculate_most_active_countries_institutions(country_codes_new, df_trec, path="../../../data/OpenAlex_TREC.json" )

In [19]:
df_trec

,PubYear,url,Authors,Title,Section,filename,filepath,ID,Countries,Classification
0,2000,http://trec.nist.gov/pubs/trec9/papers/overvie...,"[Ellen M. Voorhees, Donna Harman]",Overview of the Ninth Text REtrieval Conferenc...,Uncategorized,overview_9.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1,[],Participant
1,2000,http://trec.nist.gov/pubs/trec9/papers/trec9-c...,"[Fredric C. Gey, Aitao Chen]",TREC-9 Cross-Language Information Retrieval (E...,Uncategorized,trec9-clir-overview.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_2,[],Participant
2,2000,http://trec.nist.gov/pubs/trec9/papers/filteri...,"[Stephen E. Robertson, David A. Hull]",The TREC-9 Filtering Track Final Report.,Uncategorized,filtering_new.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_3,"[[United States, Microsoft (United States), St...",Participant
3,2000,http://trec.nist.gov/pubs/trec9/papers/t9irep.pdf,"[William R. Hersh, Paul Over]",The TREC-9 Interactive Track Report.,Uncategorized,t9irep.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_4,[],Participant
4,2000,http://trec.nist.gov/pubs/trec9/papers/liggett...,"[Walter Liggett, Chris Buckley]",Query Expansion Seen Through Return Order of R...,Uncategorized,liggett.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_5,"[[Egypt, National Institute of Standards, Walt...",Participant
...,...,...,...,...,...,...,...,...,...,...
1705,2016,https://trec.nist.gov/pubs/trec25/papers/IRIT-...,"[Gia-Hung Nguyen, Laure Soulier, Lynda Tamine,...",IRIT @ TREC 2016 Clinical Decision Support Track,Participant,IRIT-CL.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1966,"[[France, Université Toulouse - Jean Jaurès, G...",Participant
1706,2012,https://trec.nist.gov/pubs/trec21/papers/UTAus...,"[Hyun Joon Jung, Matthew Lease]",UT Austin in the TREC 2012 Crowdsourcing Track...,Participant,UTAustin.crowd.final.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1967,"[[United States, The University of Texas at Au...",Participant
1707,2007,https://trec.nist.gov/pubs/trec16/papers/umelb...,"[William Webber, Vo Ngoc Anh, Alistair Moffat]",The University of Melbourne in the Million Que...,Participant,umelbourne.ngoc-ahn.MQ.final.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1969,[],Participant
1708,2020,https://trec.nist.gov/pubs/trec29/papers/OVERV...,"[Asia J. Biega, Fernando Diaz, Michael D. Ekst...",Overview of the TREC 2020 Fair Ranking Track∗,Overview,OVERVIEW.FR.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1970,[],Overview


In [21]:
# Define concordance dicts in order to classify documents into particpant papers or overview papers

concordance_dict_trec_overview = {}
concordance_dict_trec_overview["Uncategorized"] = ["Uncategorized"]
concordance_dict_trec_overview["Participant"] = ["Papers", "Participant", "Other Papers", "Participant Papers", "Uncategorized"]
concordance_dict_trec_overview["Overview"] = ["Track Overviews", "Overview Papers", "Overview"]


In [22]:
def replace_values(val):
    for key, value_list in concordance_dict_trec_overview.items():
        if val in value_list:
            return key
    return val

# Apply the concordance dict in order to unify the classification of column "Section"
df_trec['Classification'] = df_trec['Section'].apply(replace_values)

In [28]:
pattern = r'(trec\d{1,2})'

In [31]:
# Create paths to the location of the XML-transformed TREC PDFs from GROBID

import re
paths = []
for i, j in df_trec.iterrows():
    match = re.search(pattern, j["filepath"])
    
    if j["Classification"] == "Uncategorized":
        path = "./resources/XML_TREC/" +  match.group(0) + "/" + j["filename"].replace(".pdf", ".tei.xml")
    else:
        path = "./resources/XML_TREC/" +  match.group(0) + "/" + j["Classification"] + "/" + j["filename"].replace(".pdf", ".tei.xml")


    paths.append(path)

In [32]:
df_trec_test["paths"] = paths

In [33]:
from pathlib import Path

# Function to extract the required informations like author names, institutions and corresponding countries to the institutions from the XML-files transformed with GROBID
def extract_authors_info(path):
    authors_info = []
    file_path = Path(path)
    tree = ET.parse(file_path)
    root = tree.getroot()
    ns = {'ns0': 'http://www.tei-c.org/ns/1.0'}

    source_desc_elem = root.find('.//ns0:sourceDesc', namespaces=ns)
    if source_desc_elem is None:
        return authors_info
    
    for author_elem in source_desc_elem.findall('.//ns0:author', namespaces=ns):
        
        forename_elem = author_elem.find('./ns0:persName/ns0:forename', namespaces=ns)
        if forename_elem is not None:
            forename = forename_elem.text
        else:
            forename = ""
        
        surname_elem = author_elem.find('./ns0:persName/ns0:surname', namespaces=ns)
        if surname_elem is not None:
            surname = surname_elem.text
        else:
            surname = ""
        
        author_name = f"{forename} {surname}"
        
        # Extract affiliations of the author
        for aff_elem in author_elem.findall('./ns0:affiliation', namespaces=ns):
            org_name_elem = aff_elem.find('./ns0:orgName', namespaces=ns)
            if org_name_elem is not None:
                org_name = org_name_elem.text
            else:
                org_name = ""
            address_elem = aff_elem.find('./ns0:address', namespaces=ns)
            if address_elem is not None:
                country_elem = address_elem.find('./ns0:country', namespaces=ns)
                if country_elem is not None and country_elem.text is not None and org_name != "":
                    authors_info.append([country_elem.text, org_name, author_name])
    return authors_info

In [34]:
import xml.etree.ElementTree as ET

list_metadata_grobid_extract = []

# Extracting the metadata based on the paths 

for i,j in df_trec.iterrows():
    try:
        author_infos = extract_authors_info(j["paths"])
        list_metadata_grobid_extract.append(author_infos)
    except:
        list_metadata_grobid_extract.append([])

In [35]:
df_trec_test["grobid"] = liste_metadata_grobid_extract

In [36]:
df_trec_test

,PubYear,url,Authors,Title,Section,filename,filepath,ID,Countries,Classification,paths,grobid
0,2000,http://trec.nist.gov/pubs/trec9/papers/overvie...,"[Ellen M. Voorhees, Donna Harman]",Overview of the Ninth Text REtrieval Conferenc...,Uncategorized,overview_9.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1,[],Uncategorized,./resources/XML_TREC/trec25/overview_9.tei.xml,[]
1,2000,http://trec.nist.gov/pubs/trec9/papers/trec9-c...,"[Fredric C. Gey, Aitao Chen]",TREC-9 Cross-Language Information Retrieval (E...,Uncategorized,trec9-clir-overview.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_2,[],Uncategorized,./resources/XML_TREC/trec11/trec9-clir-overvie...,[]
2,2000,http://trec.nist.gov/pubs/trec9/papers/filteri...,"[Stephen E. Robertson, David A. Hull]",The TREC-9 Filtering Track Final Report.,Uncategorized,filtering_new.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_3,"[[United States, Microsoft (United States), St...",Uncategorized,./resources/XML_TREC/trec28/filtering_new.tei.xml,[]
3,2000,http://trec.nist.gov/pubs/trec9/papers/t9irep.pdf,"[William R. Hersh, Paul Over]",The TREC-9 Interactive Track Report.,Uncategorized,t9irep.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_4,[],Uncategorized,./resources/XML_TREC/trec21/t9irep.tei.xml,[]
4,2000,http://trec.nist.gov/pubs/trec9/papers/liggett...,"[Walter Liggett, Chris Buckley]",Query Expansion Seen Through Return Order of R...,Uncategorized,liggett.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_5,"[[Egypt, National Institute of Standards, Walt...",Uncategorized,./resources/XML_TREC/trec10/liggett.tei.xml,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
1705,2016,https://trec.nist.gov/pubs/trec25/papers/IRIT-...,"[Gia-Hung Nguyen, Laure Soulier, Lynda Tamine,...",IRIT @ TREC 2016 Clinical Decision Support Track,Participant,IRIT-CL.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1966,"[[France, Université Toulouse - Jean Jaurès, G...",Participant,./resources/XML_TREC/trec25/Participant/IRIT-C...,"[[France, France, IRIT, Gia-Hung Nguyen], [Fra..."
1706,2012,https://trec.nist.gov/pubs/trec21/papers/UTAus...,"[Hyun Joon Jung, Matthew Lease]",UT Austin in the TREC 2012 Crowdsourcing Track...,Participant,UTAustin.crowd.final.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1967,"[[United States, The University of Texas at Au...",Participant,./resources/XML_TREC/trec21/Participant/UTAust...,[]
1707,2007,https://trec.nist.gov/pubs/trec16/papers/umelb...,"[William Webber, Vo Ngoc Anh, Alistair Moffat]",The University of Melbourne in the Million Que...,Participant,umelbourne.ngoc-ahn.MQ.final.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1969,[],Participant,./resources/XML_TREC/trec16/Participant/umelbo...,"[[Australia, Department of Computer Science an..."
1708,2020,https://trec.nist.gov/pubs/trec29/papers/OVERV...,"[Asia J. Biega, Fernando Diaz, Michael D. Ekst...",Overview of the TREC 2020 Fair Ranking Track∗,Overview,OVERVIEW.FR.pdf,D:/Studium/Masterarbeit/PythonProject/data/PDF...,trec_1970,[],Overview,./resources/XML_TREC/trec29/Overview/OVERVIEW....,[]


In [41]:
df_trec_test.to_parquet("../../../data/network_parquet_files/Network_preparations_TREC.parquet")